In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from skimage.feature import hog

# Define paths for each fracture type
root = 'data/'
paths = {
    'Hairline Fracture': root + 'Hairline Fracture/Train',
    'Spiral Fracture': root + 'Spiral Fracture/Train',
    'Greenstick Fracture': root + 'Greenstick fracture/Train',
    'Comminuted Fracture': root + 'Comminuted fracture/Train',
    'Fracture Dislocation': root + 'Fracture Dislocation/Train',
    'Pathological Fracture': root + 'Pathological fracture/Train',
    'Longitudinal Fracture': root + 'Longitudinal fracture/Train',
    'Oblique Fracture': root + 'Oblique fracture/Train',
    'Impacted Fracture': root + 'Impacted fracture/Train',
    'Avulsion Fracture': root + 'Avulsion fracture/Train'
}

# Load and label the dataset
def load_and_label_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

def get_dataset(paths):
    datasets = []
    for label, (fracture_type, path) in enumerate(paths.items()):
        list_ds = tf.data.Dataset.list_files(path + '/*.jpg', shuffle=False)
        labeled_ds = list_ds.map(lambda x: load_and_label_image(x, label), num_parallel_calls=tf.data.AUTOTUNE)
        datasets.append(labeled_ds)
    return datasets

train_datasets = get_dataset(paths)
train_dataset = train_datasets[0]
for dataset in train_datasets[1:]:
    train_dataset = train_dataset.concatenate(dataset)

# Shuffle the dataset for performance
train_dataset = train_dataset.shuffle(1000).prefetch(tf.data.AUTOTUNE)

# Define maximum lengths for features
HOG_FEATURE_LENGTH = 7744  # Example length for HOG features
CANNY_FEATURE_LENGTH = 65536  # Example length for Canny edge features
CONTOUR_FEATURE_LENGTH = 7 * 5  # Example length for contour features (7 Hu moments * 5 contours)
FREQUENCY_FEATURE_LENGTH = 65536  # Example length for frequency components

def pad_or_truncate(features, length):
    if len(features) > length:
        return features[:length]
    else:
        return np.pad(features, (0, length - len(features)), 'constant')

# Custom feature extraction functions
def extract_hog_features(image):
    image = (image * 255).numpy().astype('uint8')
    fd = hog(image, orientations=9, pixels_per_cell=(8, 8),
             cells_per_block=(2, 2), visualize=False, channel_axis=-1)
    return pad_or_truncate(fd, HOG_FEATURE_LENGTH)

def extract_canny_edges(image):
    image = (image * 255).numpy().astype('uint8')
    edges = cv2.Canny(image, threshold1=100, threshold2=200)
    return pad_or_truncate(edges.ravel(), CANNY_FEATURE_LENGTH)

def extract_contours(image):
    image = (image * 255).numpy().astype('uint8')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_features = []
    for contour in contours[:5]:  # Limit to the first 5 contours
        moments = cv2.moments(contour)
        hu_moments = cv2.HuMoments(moments).flatten()
        contour_features.extend(hu_moments)
    return pad_or_truncate(np.array(contour_features), CONTOUR_FEATURE_LENGTH)

def extract_frequency_components(image):
    image = (image * 255).numpy().astype('uint8')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    return pad_or_truncate(magnitude_spectrum.ravel(), FREQUENCY_FEATURE_LENGTH)

def extract_features(image, label):
    hog_features = extract_hog_features(image)
    canny_features = extract_canny_edges(image)
    contour_features = extract_contours(image)
    frequency_features = extract_frequency_components(image)
    combined_features = np.concatenate([hog_features, canny_features, contour_features, frequency_features])
    return combined_features, label

def feature_extraction(image, label):
    image, label = tf.py_function(extract_features, [image, label], [tf.float32, tf.int32])
    feature_length = HOG_FEATURE_LENGTH + CANNY_FEATURE_LENGTH + CONTOUR_FEATURE_LENGTH + FREQUENCY_FEATURE_LENGTH
    image.set_shape([feature_length])  # Set the shape to the fixed feature length
    label.set_shape([])  # Set the shape for the label
    return image, label

# Apply feature extraction
train_dataset = train_dataset.map(feature_extraction, num_parallel_calls=tf.data.AUTOTUNE)

# Batch the dataset after feature extraction
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Define and compile a simple neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(HOG_FEATURE_LENGTH + CANNY_FEATURE_LENGTH + CONTOUR_FEATURE_LENGTH + FREQUENCY_FEATURE_LENGTH,)),  # Set input_shape based on combined feature length
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # Adjust based on the number of classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10)

# Verify the shape of the feature matrix and labels
for features_batch, labels_batch in train_dataset.take(1):
    print('Features batch shape:', features_batch.shape)
    print('Labels batch shape:', labels_batch.shape)
